In [1]:
!nvidia-smi   # Volatile GPU-Util 50%以下模型不好

Sun Dec  5 20:20:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.80       Driver Version: 462.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 2080   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   53C    P8    13W /  N/A |    822MiB /  8192MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [16]:
torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x2000fef1fc8>)

## 查询可用的GPU数量

In [3]:
torch.cuda.device_count()

1

## 这两个函数允许我们在请求的GPU不存在的情况下运行代码

In [4]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## 查询张量所在设备

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

## 存储在GPU上

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

## 在指定GPU上创建一个张量

In [20]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.4422, 0.6123, 0.5901],
        [0.7742, 0.0617, 0.7845]], device='cuda:0')

## 要计算X + Y，我们需要决定在哪里执行这个操作

In [21]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


## 进行计算的数据需要在同一设备上

In [22]:
Y + Z   # GPU与CPU之间移动数据很慢

tensor([[1.4422, 1.6123, 1.5901],
        [1.7742, 1.0617, 1.7845]], device='cuda:0')

In [25]:
Z.cuda(0) is Z   # 如果已经在该设备上就不会移动而返回自身

True

## 神经网络与GPU

In [24]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [13]:
net(X)

tensor([[0.1487],
        [0.1487]], device='cuda:0', grad_fn=<AddmmBackward0>)

## 确认模型参数存储在同一个GPU上

In [14]:
net[0].weight.data.device

device(type='cuda', index=0)